In [ ]:
# 查看GPU配置
# Check GPU configuration
!nvidia-smi

#这个笔记本由于pipeline更新已经不能使用了，请使用[新版本](https://colab.research.google.com/drive/1pn1xnFfdLK63gVXDwV4zCXfVeo8c-I-0?usp=sharing)
#同时请参照新版本的[使用说明](https://github.com/Plachtaa/VITS-fast-fine-tuning/blob/main/README_ZH.md)
#不便之处敬请谅解😥

In [ ]:
#@title
#@markdown #STEP 1 (3 min)
#@markdown ##复制代码库并安装运行环境
#@markdown ##Clone repository & Build environment
! git clone https://github.com/lopezjuanma96/VITS-fast-fine-tuning.git
%cd VITS-fast-fine-tuning
!pip install -r requirements.txt
# build monotonic align
%cd monotonic_align/
!mkdir monotonic_align
!python setup.py build_ext --inplace
%cd ..
!mkdir pretrained_models
%cd pretrained_models
# download pretrained discriminator checkpoint
!wget https://huggingface.co/spaces/Plachta/VITS-Umamusume-voice-synthesizer/resolve/main/pretrained_models/D_trilingual.pth
#!wget https://huggingface.co/spaces/Plachta/VITS-Umamusume-voice-synthesizer/resolve/main/pretrained_models/G_trilingual.pth
!wget https://huggingface.co/lopezjm96/spanishVITS/resolve/main/G_spanish.pth -O G_trilingual.pth
%cd ..
# download data for fine-tuning
!wget https://huggingface.co/datasets/lopezjm96/spanish_voices/resolve/main/spanish_voices.zip
!unzip spanish_voices.zip

In [ ]:
#@title
#@markdown #STEP 2 (5~10 min)
#@markdown 如果你不需要变声器功能，可以跳过这步。  

#@markdown 运行该代码块之后会出现一个UI，在这里阅读短句从而录入你的声音。  
#@markdown 录入20句以达到最低限度的效果。录入40句以上可以保证最佳效果。  

#@markdown (No. 0~44 为中文句子)  

#@markdown If you don't need the function of converting your voice to other characters', you can skip this block 

#@markdown Running this code block will present a UI below. 
#@markdown Read the text to teach the model your voice! 
#@markdown (20 at least, for better conversion quality, 40+ is suggested)  

#@markdown (No. 45~92 are English sentences)
%run user_voice_collect.py

In [ ]:
#@title
#@markdown #STEP 2.5
#@markdown ### 可选：使用Demucs对录音进行去噪以提高转换质量 （花费约8秒一句）
#@markdown ### Optional: Denoise recorded audio for better voice conversion quality
#@markdown ### (takes about 8 seconds per .wav file on colab free GPU)
!python demucs_denoise.py --denoise_user True

In [ ]:
#@title
#@markdown #STEP 3
#@markdown 如果你不需要添加自己的角色，可以跳过这步。  

#@markdown 运行该代码块会出现一个文件上传的入口，
#@markdown 按照[README](https://github.com/Plachtaa/VITS-fast-fine-tuning/blob/main/README_ZH.md)中描述的格式上传单个zip文件。  

#@markdown 注意只能上传zip文件，上传其它压缩文件格式(.rar, .7z等)是解不出来的  

#@markdown Whisper在自动标注时出现错别字是几乎不影响标注质量的，
#@markdown 因为最后转化成音素的时候都差不多。

#@markdown If you don't want to add your characters, you can skip this block.  

#@markdown Running this codeblock will prompt you to upload a file. 
#@markdown You should upload a zip file, whose file structure should 
#@markdown be consistent as described in the [README](https://github.com/Plachtaa/VITS-fast-fine-tuning/blob/main/README_EN.md) page.
%run voice_upload.py
!unzip ./custom_character_voice/custom_character_voice.zip -d ./custom_character_voice/
!python whisper_transcribe.py

In [ ]:
%run preprocess.py
%reload_ext tensorboard
%tensorboard --logdir "./OUTPUT_MODEL"
#@markdown #STEP 4 (>=20 min)
#@markdown 开始微调模型，在20轮训练后会自动结束。
#@markdown 训练时长取决于你录入/上传的音频总数。

#@markdown 一般建议设置为10~20epochs，多了可能会过拟合。  

#@markdown 你也可以在Tensorboard中预览合成效果，若效果满意可提前停止。

#@markdown Model fine-tuning ends after 20 epochs. 
#@markdown Total time cost depends on the number of voices you recorded/uploaded.   

#@markdown 10~20 epochs is recommended because too much training may result in overfitting.

#@markdown You can also preview synthezied audio in Tensorboard, it's OK to shut down training manually if you find the quality is satisfying.
# Model checkpoints are automatically saved to ./OUTPUT_MODEL/G_latest.pth
Maximum_epochs = "20" #@param [10, 20, 30]
!python finetune_speaker.py -m "./OUTPUT_MODEL" --max_epochs "{Maximum_epochs}"

In [ ]:
#@title
#@markdown ### 微调完成后，在这里尝试效果。
#@markdown ### Try out conversion quality here after fine-tuning is finished.
!cp ./configs/modified_finetune_speaker.json ./finetune_speaker.json
%run VC_inference.py --model_dir ./OUTPUT_MODEL/G_latest.pth

In [ ]:
#@title
#@markdown ### 下载微调好的模型和config文件以便本地部署。
#@markdown ### Download fine-tuned model & config file for local inference.
!cp ./configs/modified_finetune_speaker.json ./finetune_speaker.json
%run download_model.py